<a href="https://colab.research.google.com/github/shyamsparrow/Jan-G4---NLP-Chatbot/blob/main/Chatbot_Flask_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/AIML/Capstone/Jan-G4---NLP-Chatbot

/content/drive/MyDrive/AIML/Capstone/Jan-G4---NLP-Chatbot


In [4]:
!git checkout main

Checking out files: 100% (26/26), done.
Switched to branch 'main'
Your branch is behind 'origin/main' by 9 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)


In [12]:
!git pull

Already up to date.


# Installing Libraries

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import sys  
!{sys.executable} -m pip install contractions==0.0.18
!pip install pyspellchecker
!pip install contractions
!pip install tensorflow==2.7.0 --user
quiet = True

In [8]:
%cd /content/drive/MyDrive/AIML/Capstone/Jan-G4---NLP-Chatbot/Utilities

/content/drive/MyDrive/AIML/Capstone/Jan-G4---NLP-Chatbot/Utilities


# Importing Model and Creating Predictions

In [9]:
warnings.filterwarnings("ignore")

import pickle
import os
import random
import numpy as np


from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

from NLP_text_preprocess import PreProcessing
import config

text_contents=dict()


model = load_model("/content/drive/MyDrive/AIML/Capstone/Jan-G4---NLP-Chatbot/Models/LSTM_model.h5")
    
def greet_res(text):
    bot_greet=['hi','hello','hola','hey','howdy']
    usr_greet=['hi','hey','hello','hola','greetings','wassup','whats up']
    for word in text.split():
        if word in usr_greet:
            return random.choice(bot_greet)

# bot response
def bot_ress(usr_input):
    warnings.filterwarnings("ignore")
    with open(r"parameters.pickle", "rb") as f:
          lab_en, tokenizer_data = pickle.load(f)
    pp = PreProcessing(to_lower = config.to_lower,remove_url=config.remove_url,
                       remove_time = config.remove_time,expand_contraction =config.expand_contraction,
                       remove_special_character=config.remove_special_character,
                       remove_punctuation=config.remove_punctuation,
                       remove_whitespace=config.remove_whitespace,
                       keep_alpha_numeric = False,
                       check_spelling=config.check_spelling,
                       remove_stopword=False,
                       lemmatize_word=config.lemmatize_word)

    pre_txt = pp.preprocess(usr_input)
    tok_txt = tokenizer_data.texts_to_sequences([pre_txt])
    pad_txt = pad_sequences(tok_txt, padding='post', maxlen=185)
    prediction = model.predict(pad_txt, verbose=0)
    pred_index = np.argmax(prediction,axis=1)
    result = lab_en.inverse_transform(pred_index)[0]
    bot_res = 'The Potential accident level for this accident is {}'.format(result)
    return bot_res
    
exit_list = ['exit','break','quit','see you later','chat with you later','end the chat','bye','ok bye']

def chat_res(usr_input):
    usr_input = usr_input.lower()
    #textcon.insert(END, f'User: {usr_input}'+'\n','usr')
    if usr_input in exit_list:
        return ("Ok bye! Chat with you later")
    else:
        if greet_res(usr_input) != None:
           return greet_res(usr_input)
        else:
          return bot_ress(usr_input)

In [10]:
%cd /content/drive/MyDrive/AIML/Capstone/Jan-G4---NLP-Chatbot

/content/drive/MyDrive/AIML/Capstone/Jan-G4---NLP-Chatbot


# Running Flask API

In [13]:
from flask import Flask, render_template, request

app = Flask(__name__)
app.static_folder = 'static'

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    return chat_res(userText)


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
